In [ ]:
!wget https://www.dropbox.com/s/h16vq9rab1itifs/CatDog.zip

--2024-04-14 04:17:24--  https://www.dropbox.com/s/h16vq9rab1itifs/CatDog.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/h16vq9rab1itifs/CatDog.zip [following]
--2024-04-14 04:17:24--  https://www.dropbox.com/s/raw/h16vq9rab1itifs/CatDog.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce8ac034be324b2045a137340f7.dl.dropboxusercontent.com/cd/0/inline/CQ9XyTWDBrRW4FTlfZf1D2ie5K7J1hQ3-2rSVM8ObpthVR471RgT8zr8l8Q4VFyeHqxyGlcqOyajIf-03tBPQcV7JhxhP7otapmpY5UZP93er8tY1maKQO8ArMOtQwpJ_xGy1oBZsh0GKya-skhCFis3/file# [following]
--2024-04-14 04:17:25--  https://uce8ac034be324b2045a137340f7.dl.dropboxusercontent.com/cd/0/inline/CQ9XyTWDBrRW4FTlfZf1D2ie5K7J1hQ3-2rSVM8ObpthVR471RgT8zr8l8Q4VFyeHqxyGlcqOyajIf-03tBPQcV7JhxhP7otapmpY5UZP93e

In [ ]:
!unzip CatDog.zip
!rm CatDog.zip

# Imports

In [ ]:
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
import shutil

In [ ]:
TRAIN_DIR = "/content/training_set/training_set"
d = {'cats':0,'dogs':0}

for dir in os.listdir(TRAIN_DIR):
  path = os.path.join(TRAIN_DIR,dir)
  d[dir] = len(os.listdir(path))
  print(f"path is {path}, and there are {d[dir]} images in {dir} folder")

In [ ]:
d

In [ ]:
img = "/content/training_set/training_set/dogs/dog.1004.jpg"
from skimage.io import imshow
imshow(img)
plt.show()

In [ ]:
dirs = ['train','test','val']
ROOT_DIR ='./data_dir'

if not os.path.exists(ROOT_DIR):
  for d in dirs:
    os.makedirs(os.path.join(ROOT_DIR,d))

In [ ]:
for dir in os.listdir(TRAIN_DIR):
  de = os.path.join(ROOT_DIR,'train',dir)
  os.mkdir(de)
  for img in np.random.choice( os.listdir(os.path.join(TRAIN_DIR, dir)),  int(len(os.listdir(os.path.join(TRAIN_DIR, dir)))*0.7) , replace= False ) :
    org = os.path.join(TRAIN_DIR, dir,img)
    dest = os.path.join(de,img)
    shutil.copyfile(org, dest )
    os.remove(org)

In [ ]:
for dir in os.listdir(TRAIN_DIR):
  de = os.path.join(ROOT_DIR,"val",dir)
  os.mkdir(de)
  for img in np.random.choice( os.listdir(os.path.join(TRAIN_DIR, dir)),  int(len(os.listdir(os.path.join(TRAIN_DIR, dir)))*0.5) , replace= False ) :
    org = os.path.join(TRAIN_DIR, dir,img)
    dest = os.path.join(de,img)

    shutil.copyfile(org, dest )
    os.remove(org)

In [ ]:
for dir in os.listdir(TRAIN_DIR):
  de = os.path.join(ROOT_DIR,"test",dir)
  os.mkdir(de)
  for img in np.random.choice( os.listdir(os.path.join(TRAIN_DIR, dir)),  int(len(os.listdir(os.path.join(TRAIN_DIR, dir)))), replace= False ) :
    org = os.path.join(TRAIN_DIR, dir,img)
    dest = os.path.join(de,img)

    shutil.copyfile(org, dest )
    os.remove(org)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
traindatagen = ImageDataGenerator(rescale = 1.0/255,
                                  rotation_range=0.3,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  vertical_flip=True)

In [ ]:
train_data = traindatagen.flow_from_directory(directory="/content/data_dir/train" , class_mode='binary', target_size=(224,224))

In [ ]:
valdatagen = ImageDataGenerator(rescale = 1.0/255)
val_data = valdatagen.flow_from_directory(directory="/content/data_dir/val" , class_mode='binary', target_size=(224,224))

In [ ]:
testdatagen = ImageDataGenerator(rescale = 1.0/255)
test_data = testdatagen.flow_from_directory(directory="/content/data_dir/test" , class_mode='binary', target_size=(224,224))

In [ ]:
train_data.class_indices

In [ ]:
CLASS_IDX = {j:i for i,j in train_data.class_indices.items()}
CLASS_IDX

# Build the Model

In [ ]:
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dropout,Flatten,Dense

In [ ]:
from logging import Filterer
model = Sequential()
model.add(Conv2D(filters=16,kernel_size=(3,3), activation='relu', input_shape=(224,224,3) ))
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Dropout(rate=0.25))

model.add(Flatten())

model.add(Dense(units=64, activation='relu'))
model.add(Dropout(rate=0.25))

model.add(Dense(units=1, activation='sigmoid'))

model.summary()

In [ ]:
26*26*128

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [ ]:
es =EarlyStopping(monitor='val_loss', min_delta=0.01, patience=5, verbose=0, mode='auto')
mc = ModelCheckpoint(filepath='best_mode.h5', monitor='val_loss',verbose=0, save_best_only=True)

In [ ]:
hist = model.fit_generator(generator=train_data,
                           validation_data=val_data,
                           steps_per_epoch=32,
                           epochs=30,
                           verbose=1,
                           callbacks=[es,mc])

Epoch 1/30
32/32 [==============================] - 221s 7s/step - loss: 0.7351 - accuracy: 0.5166 - val_loss: 0.6838 - val_accuracy: 0.5658
Epoch 2/30


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


32/32 [==============================] - 221s 7s/step - loss: 0.6674 - accuracy: 0.5869 - val_loss: 0.6536 - val_accuracy: 0.5983
Epoch 3/30
32/32 [==============================] - 211s 7s/step - loss: 0.6809 - accuracy: 0.5377 - val_loss: 0.6516 - val_accuracy: 0.6117
Epoch 4/30
32/32 [==============================] - 209s 7s/step - loss: 0.6776 - accuracy: 0.5910 - val_loss: 0.6623 - val_accuracy: 0.6125
Epoch 5/30
32/32 [==============================] - 214s 7s/step - loss: 0.6632 - accuracy: 0.6113 - val_loss: 0.6581 - val_accuracy: 0.6117
Epoch 6/30
32/32 [==============================] - 214s 7s/step - loss: 0.6517 - accuracy: 0.6152 - val_loss: 0.6565 - val_accuracy: 0.6550
Epoch 7/30
32/32 [==============================] - 215s 7s/step - loss: 0.6469 - accuracy: 0.6309 - val_loss: 0.6257 - val_accuracy: 0.6483
Epoch 8/30
32/32 [==============================] - 222s 7s/step - loss: 0.6573 - accuracy: 0.6309 - val_loss: 0.6232 - val_accuracy: 0.6442
Epoch 9/30
32/32 [======

## Load the model

In [ ]:
from tensorflow.keras.models import load_model
best_model = load_model("/content/best_mode.h5")

In [ ]:
#load the data from external file
# convert the image into array & resize = (224,224,3)
# Rescale into the pixel intensity range = 0 to 1
# Addition axis for batch size
# model.predict(new_image)

In [ ]:
!wget https://www.dropbox.com/s/m29kun39a6ogeif/test_cat1.jpg

In [ ]:
from tensorflow.keras.utils import load_img,img_to_array

In [ ]:
!wget https://www.dropbox.com/s/urmijmcw3ya66o6/test%20dog.jpg

In [ ]:
# !wget https://www.dropbox.com/s/urmijmcw3ya66o6/test%20dog.jpg
pil = load_img("/content/cat2.jpeg",target_size=(224,224))
image = img_to_array(pil)
image = image/255
plt.imshow(image)
plt.title("Input Image")
plt.show()
print("*************************************************************")
image = np.expand_dims(image,axis=0)
pred = int(model.predict(image) > 0.5)
print(f"Prediction from my model is {CLASS_IDX[pred]}")

In [ ]:
!wget https://www.dropbox.com/s/5syxnxil7vjaa6x/dog_test2.jpg
pil = load_img("/content/dog_test2.jpg",target_size=(224,224))
image = img_to_array(pil)
image = image/255
plt.imshow(image)
plt.title("Input Image")
plt.show()
print("*************************************************************")
image = np.expand_dims(image,axis=0)
pred = int(model.predict(image) > 0.5)
print(f"Prediction from my model is {CLASS_IDX[pred]}")

In [ ]:
# Pre-trained Model - VGG16
# Prepare the using VGG16

# Training with VGG16 Model

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
vgg_full = VGG16(input_shape=(224,224,3), include_top=True, weights='imagenet')
vgg_full.summary()

In [ ]:
vgg_full.get_config()

In [ ]:
vgg_partial = VGG16(input_shape=(224,224,3), include_top=False, weights='imagenet')
vgg_partial.summary()

In [ ]:
len(vgg_partial.layers)

In [ ]:
for layer in vgg_partial.layers[:15]:
  layer.trainable = False

for layer in vgg_partial.layers[15:]:
  layer.trainable = True

In [ ]:
from tensorflow.keras.layers import Flatten,Dense,Conv2D,Dropout

In [ ]:
final_layer.output

In [ ]:
final_layer = vgg_partial.layers[-1]
x = Flatten()(final_layer.output)
x = Dense(512,activation='relu') (x)
x = Dropout(0.3)(x)
x = Dense(1,activation='sigmoid')(x)

In [ ]:
from tensorflow.keras.models import Model
final_model = Model(vgg_partial.input, x)

In [ ]:
final_model.summary()

In [ ]:
from tensorflow.keras.layers import GlobalMaxPool2D

In [ ]:
final_layer = vgg_partial.layers[-1]
x = GlobalMaxPool2D()(final_layer.output)
x = Dense(512,activation='relu') (x)
x = Dropout(0.5)(x)
x = Dense(1,activation='sigmoid')(x)

final_model = Model(vgg_partial.input, x)

final_model.summary()

In [ ]:
final_model.compile(loss='binary_crossentropy',
                    optimizer = tf.keras.optimizers.SGD(learning_rate=1e-4, momentum=0.9),
                    metrics=['accuracy'])

In [ ]:
history = final_model.fit(train_data,
                                    steps_per_epoch=100,
                                    epochs=10,
                                    validation_data=val_data,
                                    validation_steps=100,
                                    callbacks=[mc])

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import load_img,img_to_array
best_model = load_model("/content/best_mode.h5")

!wget https://www.dropbox.com/s/urmijmcw3ya66o6/test%20dog.jpg
pil = load_img("/content/test dog.jpg",target_size=(224,224))
image = img_to_array(pil)
image = image/255
plt.imshow(image)
plt.title("Input Image")
plt.show()
print("*************************************************************")
image = np.expand_dims(image,axis=0)
pred = int(model.predict(image) > 0.5)
print(f"Prediction from my model is {CLASS_IDX[pred]}")

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import load_img,img_to_array
best_model = load_model("/content/best_mode.h5")

pil = load_img("/content/dog2.jpg",target_size=(224,224))
image = img_to_array(pil)
image = image/255
plt.imshow(image)
plt.title("Input Image")
plt.show()
print("*************************************************************")
image = np.expand_dims(image,axis=0)
pred = int(model.predict(image) > 0.5)
print(f"Prediction from my model is {CLASS_IDX[pred]}")

In [ ]:
# Learn more on GoogleNet/Resnet - Architecture

In [ ]:
# Neural Network --> Sequential - one
# GlobalMaxPool2D Layer
# difference between sparse categorial vs categorial entropy